In [2]:
import ee,os,glob,geemap
import pandas as pd, numpy as np
# authenticate to the Earth Engine API
#ee.Authenticate()
ee.Initialize()

In [26]:
path=r"C:\Users\hafez\MSU\Research\git\pace\CLAP"
os.chdir(path)
river=pd.read_csv('locationLandsat.csv',sep='\t')
river

,No,latitude,longitude
0,1,37.864444,-76.762222
1,2,37.504444,-76.787500
2,3,37.221389,-76.893611


In [6]:
# read .geojson file
roi=geemap.geojson_to_ee(r"C:\Users\hafez\MSU\Research\git\pace\CLAP\river_points.geojson")
path=r"C:\Users\hafez\MSU\Research\git\pace\CLAP"
os.chdir(path)


# Define cloud cover threshold
cloudCoverThreshold = 20

# Define the date range
startDate = '2024-05-22'
endDate = '2025-05-23'

# Function to apply scaling factors
def apply_scale_factors(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

# Filter the Landsat 8 collection
landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(roi) \
    .filterDate(startDate, endDate) \
    .filter(ee.Filter.lte('CLOUD_COVER', cloudCoverThreshold)) \
    .map(apply_scale_factors)

# Filter the Landsat 9 collection
landsat9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2') \
    .filterBounds(roi) \
    .filterDate(startDate, endDate) \
    .filter(ee.Filter.lte('CLOUD_COVER', cloudCoverThreshold)) \
    .map(apply_scale_factors)

# Merge the Landsat 8 and Landsat 9 collections
merged_collection = landsat8.merge(landsat9).mean()

bands = ['SR_B1','SR_B2','SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10']
out_csv = r'C:/Users/hafez/MSU/Research/git/pace/CLAP/landsat_coast.csv'
# extract point 
geemap.extract_values_to_points(roi, merged_collection.select(bands), out_csv, scale=10)
# red- green/ red+green



Generating URL ...
Please wait ...
Data downloaded to C:\Users\hafez\MSU\Research\git\pace\CLAP\landsat_coast.csv


In [34]:
path=r"C:\Users\hafez\MSU\Research\git\pace\CLAP"
os.chdir(path)
river=pd.read_csv('locationLandsat.csv',sep='\t')

# Define a region of interest (ROI)
longitude = river['longitude'][0]
latitude = river['latitude'][0]
# Define a point of interest (POI) using the coordinates from the river dictionary
riverd = {'longitude': longitude, 'latitude': latitude}  # Replace with actual longitude and latitude values
roi = ee.Geometry.Point([riverd['longitude'], riverd['latitude']])
# Define cloud cover threshold
cloudCoverThreshold = 20

# Define the date range
startDate = '2024-05-22'
endDate = '2025-05-23'

# Function to apply scaling factors
def apply_scale_factors(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

# Filter the Landsat 8 collection
landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(roi) \
    .filterDate(startDate, endDate) \
    .filter(ee.Filter.lte('CLOUD_COVER', cloudCoverThreshold)) \
    .map(apply_scale_factors)

# Filter the Landsat 9 collection
landsat9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2') \
    .filterBounds(roi) \
    .filterDate(startDate, endDate) \
    .filter(ee.Filter.lte('CLOUD_COVER', cloudCoverThreshold)) \
    .map(apply_scale_factors)

# Merge the Landsat 8 and Landsat 9 collections
merged_collection = landsat8.merge(landsat9).mean()

# to ee
points= geemap.df_to_ee(river)
bands = ['SR_B1','SR_B2','SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10']
out_csv = r'C:/Users/hafez/MSU/Research/git/pace/CLAP/landsat.csv'
# extract point 
geemap.extract_values_to_points(points, merged_collection.select(bands), out_csv, scale=10)
# red- green/ red+green


Generating URL ...
Please wait ...
Data downloaded to C:\Users\hafez\MSU\Research\git\pace\CLAP\landsat.csv


In [7]:

#out_csv = r'C:/Users/hafez/MSU/Research/git/pace/CLAP/landsat.csv'
# convert to Rrs
df = pd.read_csv(out_csv)
# divide by pi to B1 to B7 _new column Rrs1 to Rrs7
for i in range(1,8):
    df['Rrs'+str(i)] = df['SR_B'+str(i)]/np.pi
# save to csv
df.to_csv(out_csv, index=False)

In [5]:
from MDN import image_estimates, get_tile_data, get_sensor_bands

ImportError: This version of TensorFlow Probability requires TensorFlow version >= 2.16; Detected an installation of version 2.15.0. Please upgrade TensorFlow to proceed.